In [4]:
#Variables globales
import pandas
import re
import glob
import os

Temporada = [['1','Verano'],
['2','Primavera'],
['3','Otono'],
['4','Invierno'],
['5','Cualquiera']]

Tipo = [['1','Desayuno'],
['2','Postre'],
['3','Principal']]

Compatibilidad = [['1','AptoVegano'],
['2','AptoVegetariano'],
['3','NoAptoV']]

Coccion = [['1','Plancha'],
['2','Hervido'],
['3','Horneado'],
['4','Frito'],
['5','Estofado'],
['6','Crudo']]

TipoIngrediente = [['1','Cereal'],
['2','Legumbre'],
['3','Fruta'],
['4','Verdura'],
['5','Carne'],
['6','Pescado'],
['7','FrutoSeco'],
['8','Lacteo']]

listaNutrientes = [['AcidoPantotenico','Pantothenic acid'],
['AcidosGrasosSaturados','Saturated fatty acids'],
['AcidosGrasosTrans','Fatty acids, total trans'],
['Agua','Water'],
['Azucares','Sugars'],
['Calcio','Calcium, Ca'],
['CarbohidratosTotales','Carbohydrate'],
['Cobalamina','Vitamin B12'],
['Cobre','Copper, Cu'],
['Colina','Choline'],
['Calorias','Calories'],
['FibrasAlimenticias','Fiber'],
['Folato','Folate'],
['Fosforo','Phosphorus, P'],
['GrasaTotal','Fat'],
['Hierro','Iron, Fe'],
['Magnesio','Magnesium, Mg'],
['Manganeso','Manganese, Mn'],
['Niacina','Niacin'],
['Potasio','Potassium, K'],
['Proteinas','Protein'],
['Riboflavina','Riboflavin'],
['Selenio','Selenium, Se'],
['Tiamina','Thiamin'],
['VitaminaA','Vitamin A'],
['VitaminaB6','Vitamin B6'],
['VitaminaC','Vitamin C'],
['VitaminaD','Vitamin D'],
['VitaminaE','Vitamin E'],
['VitaminaK','Vitamin K'],
['Zinc','Zinc, Zn']]

In [53]:
def convertCSV(file):
    f = open(file, "r")
    f.readline()
    f.readline()
    lNombre = f.readline()
    lPeso = f.readline().split()
    NombrePlato = re.sub('[^0-9a-zA-Z ]+', '', lNombre).strip()
    for l in reversed(lPeso):
        x = re.sub('[^0-9.]+','',l)
        if x != '':
            Peso = x
            break
    finalFile = open('tmp.csv',"w") 
    L = f.readlines()
    finalFile.writelines(L) 
    finalFile.close()
    f.close()
    return NombrePlato, Peso

In [6]:
def processCSV(file):
    df = pandas.read_csv('tmp.csv',names = ['Nutrient', 'Amount', 'Unit'])
    for nutriente in listaNutrientes:
        find = False
        for index, row in df.iterrows():
            if row['Nutrient'] == nutriente[1]:
                find = True
                monto = float(row['Amount'])
                if row['Unit'].strip() == "g":
                    monto *= 1000.0
                elif row['Unit'].strip() == 'mcg':
                    monto /= 1000000.0
                elif row['Nutrient'] == 'Vitamin A':
                    monto *= 0.3
                    monto /= 1000.0
                elif row['Nutrient'] == 'Vitamin D':
                    monto *= 0.025 
                    monto /= 1000.0
                file.write("(" + nutriente[0] + " %8.8f)" % monto)
        if find is False:
            file.write("(" + nutriente[0] + " " + str(0.0) + ")")
        if nutriente != listaNutrientes[-1]:
            file.write("\n")

In [7]:
def choose(options):
    if len(options) == 0:
        return ''
    s = ""
    for x in options:
        s += x[0] + " " + x[1] + "\n"
    s += "Escribe el numero (Salir 0): "
    while True:
        var = input(s)
        if var == '0':
            return ''
        for x in options:
            if var == x[0]:
                return x[1]

In [15]:
def choose2(options, hint):
    if len(options) == 0:
        return '',''
    s = ""
    for x in options:
        s += x[0] + " " + x[1] + "\n"
    s += "Plato: " + hint + "\nEscribe el numero (Salir 0): "
    while True:
        var = input(s)
        if var == '0':
            return '',''
        for x in options:
            if var == x[0]:
                return x[1],x

In [45]:
#Modulo de añadir ingredientes al .csv
def createIngredients():
    ingredientes = open("ingredientes.csv","a")
    while True:
        NombreIngrediente = input("Introduce nombre de ingrediente (Salir 0): ")
        if NombreIngrediente == '0':
            break
        TipoIngredienteVar = choose(TipoIngrediente)
        ingredientes.write('"' + NombreIngrediente + '", '+ TipoIngredienteVar + '\n')
        if input('Salir (0): ') == '0':
            break
    ingredientes.close()

In [44]:
#Modulo de creacion de instancias de ingredientes 
def createInstancesIngredients():
    ingredientes = open("ingredientes.txt","w")
    ingredientesDir = open('ingredientesDir.txt','w')
    df = pandas.read_csv("ingredientes.csv",names = ['NombreIngrediente', 'TipoIngrediente'])
    for index, row in df.iterrows():
        ingDir = '[' + re.sub('[^0-9a-zA-Z]+', '_', row['NombreIngrediente']) + ']'
        ingredientesDir.write(ingDir + '\n')
        ingredientes.write('(' + ingDir + ' of Ingrediente\n\n')
        ingredientes.write('(NombreIngrediente "' + row['NombreIngrediente'].strip() + '")\n')
        ingredientes.write("(TipoIngrediente " + row['TipoIngrediente'].strip() + "))\n\n")
    ingredientes.close()
    ingredientesDir.close()

In [18]:
#Modulo de creacion de instancias de platos con ingredientes
def createInstancesPlatesWithIngredients():
    platos = open("platos.txt","r")
    L = platos.readlines()
    platos = open("platosIngredientes.txt","w")
    ingredientesDir = open('ingredientesDir.txt','r')
    ingredientesDir = ingredientesDir.readlines()
    lIngredientes = []
    i = 1
    for ingrediente in ingredientesDir:
        temp = len(ingrediente)
        cIngrediente = ingrediente[:temp - 1]
        lIngredientes.append([str(i),cIngrediente])
        i += 1
    for plato in L:
        platos.write(plato)
        if plato.startswith('(NombrePlato') is True:
            platos.write('(Ingredientes')
            ingDisponibles = lIngredientes.copy()
            ingElegidos = []
            while True:
                print(plato)
                text,option = choose2(ingDisponibles, plato)
                if text is '':
                    break
                platos.write('\n' + text)
                ingDisponibles.remove(option)
                ingElegidos.append(option)
                if len(ingDisponibles) == 0 or input('Salir (0): ') == '0':
                    break
            platos.write(')\n')
            ingPrincipal = choose(ingElegidos)
            if ingPrincipal is not '':
                platos.write('(IngredientePrincipal ' + ingPrincipal + ')\n')
    platos.close()

In [55]:
#Modulo de creacion de instancias de platos sin ingredientes
def createInstancesPlates():
    filenames = glob.glob("platos/export*.csv")
    platos = open("platos.txt","w")
    for file in filenames:
        NombrePlato, Peso = convertCSV(file)
        print("\n" + NombrePlato + ' (' + Peso + ' g)')
        platos.write('([' + re.sub('[^0-9a-zA-Z]+', '_', NombrePlato) + '] of Plato\n\n')
        platos.write('(NombrePlato "' + NombrePlato + '")\n')
        platos.write("(Peso " + Peso + ")\n")
        platos.write("(Temporada " + choose(Temporada) + ")\n")
        platos.write("(Tipo " + choose(Tipo) + ")\n")
        platos.write("(Compatibilidad " + choose(Compatibilidad) + ")\n")
        platos.write("(Coccion " + choose(Coccion) + ")\n")
        processCSV(platos)
        platos.write(")\n\n")
    os.remove('tmp.csv')
    platos.close()

In [41]:
def convertPounds():
    pound = 453.592
    start = open("convert.pins","r")
    L = start.readlines()
    file = open("convert2.pins","w")
    for line in L:
        if line.endswith('Plato\n'):
            convert = False
            if input(line + '\nConvertir (1): ') == '1':
                convert = True
        x = line.split()
        if convert and len(x)>0:
            if line.startswith('(Peso'):
                peso = float(re.sub('[^0-9.]+','',x[-1]))
                file.write(x[0] + " " + str(pound) +  ")\n")
                continue
            if re.sub('[^0-9.]+','',x[-1]) != '':
                monto = float(re.sub('[^0-9.]+','',x[-1]))*pound/peso
                if x[0] != '(Zinc':
                    file.write(x[0] + " %8.8f)\n" % monto)
                else:
                    file.write(x[0] + " %8.8f))\n" % monto)
            else:
                file.write(line)
        else:
            file.write(line)
    file.close()

In [64]:
#Modulo de creacion de instances.pins
if input('Añadir platos (1), Continuar (0): ') == '1':
    createInstancesPlates()
if input('Añadir ingredientes (1), Continuar (0): ') == '1':
    createIngredients()
createInstancesIngredients()
if input('Añadir ingredientes a platos (1), Continuar (0): ') == '1':
    createInstancesPlatesWithIngredients()
platos = open("platosIngredientes.txt","r")
L1 = platos.readlines()
ingredientes = open("ingredientes.txt","r")
L2 = ingredientes.readlines()
instances = open("instances.pins","w")
instances.writelines(L1) 
instances.write('\n')
instances.writelines(L2)
os.remove('ingredientesDir.txt')
os.remove('ingredientes.txt')
platos.close()
ingredientes.close()
instances.close()

Añadir platos (1), Continuar (0): 0
Añadir ingredientes (1), Continuar (0): 0
Añadir ingredientes a platos (1), Continuar (0): 0


In [7]:
file = open("extra.pddl","w")
file.write('(define (problem basic_project) (:domain software_project)\n')
file.write('(:objects\n')
numP = input('Numero de programadores: ')
numT = input('Numero de tareas: ')
file.write('\t') 
for i in range(int(numP)):
    file.write('p' + str(i) + ' ')
file.write('- programador\n')
file.write('\t')  
for i in range(int(numT)):
    file.write('t' + str(i) + ' ')
file.write('- tarea\n')
file.write(')\n')

Numero de programadores: 5
Numero de tareas: 5


2